In [4]:
from google.colab import files
uploaded = files.upload()


Saving biblioteca.db to biblioteca (1).db


In [5]:
import os
print(os.listdir())


['.config', 'biblioteca (1).db', 'biblioteca.db', 'sample_data']


** Estructura del Proyecto **

bibilioteca/

│

├── src/                  # Código fuente de la aplicación

│   ├── biblioteca.py     # Clase Biblioteca y métodos

│   └── otro_modulo.py    # Otros módulos si los hay

│

├── tests/                # Pruebas unitarias

│   ├── test_biblioteca.py

│   └── test_otro_modulo.py

│

├── data/                 # Base de datos o archivos de datos

│   └── biblioteca.db

│

└── test_suite.py         # Batería de pruebas


In [3]:
import sqlite3
import pandas as pd

# Conexión a la base
conn = sqlite3.connect('biblioteca.db')

# Leer la tabla libros en un DataFrame
cursor = conn.cursor() # Se requiere de un objeto cursor para hacer consultas
cursor.execute("SELECT * FROM libros;")
# Obtenemos todos los resultados
filas = cursor.fetchall()
# Convertimos el resultado a un DataFrame
columnas = [desc[0] for desc in cursor.description]
df = pd.DataFrame(filas, columns=columnas)
print(df)


OperationalError: no such table: libros

```
# bibiloteca.py

```

In [ ]:
import sqlite3

class Biblioteca:
    def __init__(self, conexion):
        ''' Inicializar un objeto biblioteca se debe recibir la conexión'''
        # Conexión a la BD
        self.conn = conexion
        # El objeto cursor permite hacer las consultas a la BD
        self.cursor = self.conn.cursor()

    def agregar_libro(self, titulo, autor, disponible=1):
        #Antes de insertar el libro verifica si ya existe en la BD
        self.cursor.execute("SELECT * FROM libros WHERE titulo=?", (titulo,))
        #Si ya existe lanza una excepción y aquí termina el método
        if self.cursor.fetchone():
            raise ValueError(f"El libro '{titulo}' ya está registrado.")
        #Si el libro no existe lo inserta en la BD
        self.cursor.execute(
            "INSERT INTO libros (titulo, autor, disponible) VALUES (?, ?, ?)",
            (titulo, autor, disponible)
        )
        #Ejecuta los cambios
        self.conn.commit()

    def prestar_libro(self, titulo):
        #Verifica que el libro este disponible
        self.cursor.execute("SELECT disponible FROM libros WHERE titulo=?", (titulo,))
        #Obtiene el primer resultado
        fila = self.cursor.fetchone()
        #El libro no está registrado, lanza una exepción y aqui termina el método
        if not fila:
            raise ValueError(f"El libro '{titulo}' no existe.")
        #El libro está registrado pero no disponible(=0), lanza una exepción y aqui termina el método
        if fila[0] == 0:
            raise ValueError(f"El libro '{titulo}' no está disponible.")
        #El libro está disponible, actualiza el campo disponible
        self.cursor.execute("UPDATE libros SET disponible=0 WHERE titulo=?", (titulo,))
        self.conn.commit()

    def contar_disponibles(self):
        #Cuenta cuantos libros de la BD estpan disponibles(=1)
        self.cursor.execute("SELECT COUNT(*) FROM libros WHERE disponible=1")
        return self.cursor.fetchone()[0]




```
test_biblioteca.py
```



In [6]:
import unittest

class TestBiblioteca(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        """Se conecta a la BD y crea una instancia de Bibilioteca para todas la pruebas"""


    @classmethod
    def tearDownClass(cls):
        """Cerrar la conexión al finalizar todos los tests"""


    def setUp(self):
        """Crea libros antes de cada test"""


    def tearDown(self):
        """Limpiar libros antes de cada test"""



    # ------------------------------------------
    # Test contar disponibles
    # ------------------------------------------
    def test_contar_disponibles(self):
        self.biblio.agregar_libro("Libro 1", "Autor 1", 1)
        self.assertEqual(self.biblio.contar_disponibles(), 1)

    # ------------------------------------------
    # Test de excepciones con regex
    # ------------------------------------------
    def test_agregar_libro_duplicado(self):
        self.biblio.agregar_libro("Libro Duplicado", "Autor X")
        with self.assertRaisesRegex(ValueError, r"ya está registrado"):
            self.biblio.agregar_libro("Libro Duplicado", "Autor X")

    #def test_prestar_libro_no_disponible(self):

    #def test_prestar_libro_inexistente(self):




```
test_suite.py
```



In [ ]:
#def suite_completa():



test_agregar_libro_duplicado (__main__.TestBiblioteca.test_agregar_libro_duplicado) ... ok
test_contar_disponibles (__main__.TestBiblioteca.test_contar_disponibles) ... ok
test_prestar_libro_inexistente (__main__.TestBiblioteca.test_prestar_libro_inexistente) ... ok
test_prestar_libro_no_disponible (__main__.TestBiblioteca.test_prestar_libro_no_disponible) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.096s

OK
